# 🏷️ Lab 2: Agent Taxonomy & Classification
## Module 1 - Building Agents with Different Autonomy Levels

**Duration:** 25 minutes

**Objectives:**
- Implement agents with different autonomy levels
- Understand tool permission patterns
- See how the same query produces different behaviors

**Banking Scenario:** Customer service agents with varying capabilities

---

In [ ]:
!pip install openai -q

In [ ]:
import os
import json

# =============================================================================
# GOOGLE COLAB SETUP
# =============================================================================
# Add these secrets in Colab (click 🔑 icon in left sidebar):
#   - AZURE_OPENAI_KEY: Your API key
#   - AZURE_OPENAI_ENDPOINT: Your endpoint (e.g., https://xxx.openai.azure.com/)
#   - AZURE_OPENAI_DEPLOYMENT: Your model deployment name (e.g., gpt-4o, gpt-35-turbo)
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"  # Default, will be overridden by secret if available

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    
    # Get deployment name (model name in Azure)
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        MODEL_NAME = "gpt-4o"
    
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print("✅ Credentials loaded from Colab secrets")
        print(f"   Endpoint: {AZURE_OPENAI_ENDPOINT[:40]}...")
        print(f"   Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing credentials")
except Exception as e:
    print(f"⚠️ Could not load Colab secrets: {e}")
    print("   Running in DEMO MODE")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Azure OpenAI client ready")

## Part 1: Define Tools with Different Permission Levels

In [ ]:
# Level 1: Read-Only Tools
tools_read_only = [
    {
        "type": "function",
        "function": {
            "name": "get_account_balance",
            "description": "Get current account balance for a customer",
            "parameters": {
                "type": "object",
                "properties": {
                    "account_id": {"type": "string", "description": "Account ID"}
                },
                "required": ["account_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_transaction_history",
            "description": "Get recent transactions for an account",
            "parameters": {
                "type": "object",
                "properties": {
                    "account_id": {"type": "string"},
                    "days": {"type": "integer", "default": 30}
                },
                "required": ["account_id"]
            }
        }
    }
]

# Level 2: Read + Write (Non-Financial)
tools_read_write = tools_read_only + [
    {
        "type": "function",
        "function": {
            "name": "create_support_ticket",
            "description": "Create a support ticket for the customer",
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_id": {"type": "string"},
                    "issue_type": {"type": "string", "enum": ["dispute", "inquiry", "complaint"]},
                    "description": {"type": "string"},
                    "priority": {"type": "string", "enum": ["low", "medium", "high"]}
                },
                "required": ["customer_id", "issue_type", "description"]
            }
        }
    }
]

# Level 3: Read + Write + Financial (with limits)
tools_financial = tools_read_write + [
    {
        "type": "function",
        "function": {
            "name": "initiate_dispute",
            "description": "Initiate a transaction dispute (max $500 auto-approval)",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {"type": "string"},
                    "reason": {"type": "string"},
                    "amount": {"type": "number"}
                },
                "required": ["transaction_id", "reason", "amount"]
            }
        }
    }
]

print("✅ Tools defined:")
print(f"   Level 1 (Read-Only): {len(tools_read_only)} tools")
print(f"   Level 2 (Read+Write): {len(tools_read_write)} tools")
print(f"   Level 3 (Financial): {len(tools_financial)} tools")

## Part 2: Create Agents with Different Autonomy Levels

In [ ]:
def create_agent(level: int):
    """Create an agent with specified autonomy level"""
    configs = {
        1: {
            "name": "Informer Agent",
            "system_prompt": """You are a banking information assistant (Level 1 - Informer).
You can ONLY provide information by looking up account data.
You CANNOT make any changes, create tickets, or take actions.
If asked to do something beyond information lookup, politely explain your limitations.""",
            "tools": tools_read_only
        },
        2: {
            "name": "Advisor Agent",
            "system_prompt": """You are a banking advisor (Level 2 - Advisor).
You can look up information AND create support tickets.
You can make recommendations but cannot execute financial transactions.
Always explain your reasoning when making recommendations.""",
            "tools": tools_read_write
        },
        3: {
            "name": "Operator Agent",
            "system_prompt": """You are a banking operator (Level 3 - Operator).
You can look up information, create tickets, AND initiate disputes.
For disputes over $500, you must create a ticket for human review instead.
Always verify the customer's concern before taking action.""",
            "tools": tools_financial
        }
    }
    return configs[level]

# Simulated tool execution
def execute_tool(name: str, args: dict) -> dict:
    responses = {
        "get_account_balance": {"account_id": args.get("account_id"), "balance": 5432.10, "currency": "USD"},
        "get_transaction_history": {
            "transactions": [
                {"id": "TXN-001", "amount": -299.99, "merchant": "AMZN MKTP US", "date": "2024-01-15"},
                {"id": "TXN-002", "amount": -45.00, "merchant": "UBER TRIP", "date": "2024-01-14"},
                {"id": "TXN-003", "amount": 2500.00, "merchant": "PAYROLL", "date": "2024-01-12"}
            ]
        },
        "create_support_ticket": {"ticket_id": "TKT-12345", "status": "created", "priority": args.get("priority", "medium")},
        "initiate_dispute": {"dispute_id": "DSP-67890", "status": "initiated", "amount": args.get("amount")}
    }
    return responses.get(name, {"error": "Unknown tool"})

In [ ]:
def run_demo_agent(level: int, query: str) -> dict:
    """Demo mode: Simulate agent behavior without API calls"""
    agent = create_agent(level)
    print(f"\n{'='*60}")
    print(f"🤖 {agent['name']} (Level {level}) [DEMO MODE]")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    # Simulate different behaviors based on level
    demo_responses = {
        1: {
            "tools_called": ["get_transaction_history"],
            "response": """I found the transaction you mentioned - a $299.99 charge from AMZN MKTP US on 2024-01-15.

As an Informer Agent, I can only look up information. I cannot initiate disputes or create tickets.

To dispute this charge, please contact our support team or speak with a higher-level agent."""
        },
        2: {
            "tools_called": ["get_transaction_history", "create_support_ticket"],
            "response": """I found the suspicious transaction and have created a support ticket for you.

**Transaction Found:** $299.99 charge from AMZN MKTP US (TXN-001)
**Ticket Created:** TKT-12345 (Priority: High)

Our fraud team will review this within 24-48 hours. As an Advisor, I cannot initiate the dispute directly, but the ticket ensures it gets proper attention."""
        },
        3: {
            "tools_called": ["get_transaction_history", "initiate_dispute"],
            "response": """I've investigated and initiated a dispute for you.

**Transaction:** $299.99 charge from AMZN MKTP US (TXN-001)
**Dispute ID:** DSP-67890
**Status:** Initiated

Since the amount is under $500, I was able to auto-approve this dispute. You should see a provisional credit within 1-2 business days while we investigate."""
        }
    }
    
    result = demo_responses[level]
    for tool in result["tools_called"]:
        print(f"   🔧 Calling: {tool}")
    
    print(f"\n📝 Response:")
    print(result["response"])
    print(f"\n🔧 Tools called: {result['tools_called']}")
    
    return result

In [ ]:
def run_agent(level: int, query: str) -> dict:
    """Run an agent at specified level with a query"""
    
    # Use demo mode if no API connection
    if DEMO_MODE or client is None:
        return run_demo_agent(level, query)
    
    agent = create_agent(level)
    print(f"\n{'='*60}")
    print(f"🤖 {agent['name']} (Level {level})")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    messages = [
        {"role": "system", "content": agent["system_prompt"]},
        {"role": "user", "content": query}
    ]
    
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            tools=agent["tools"] if agent["tools"] else None
        )
        
        msg = response.choices[0].message
        tools_called = []
        
        if msg.tool_calls:
            for tc in msg.tool_calls:
                tool_name = tc.function.name
                tool_args = json.loads(tc.function.arguments)
                tools_called.append(tool_name)
                print(f"   🔧 Calling: {tool_name}")
                
                result = execute_tool(tool_name, tool_args)
                messages.append(msg)
                messages.append({"role": "tool", "tool_call_id": tc.id, "content": json.dumps(result)})
            
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages
            )
            msg = response.choices[0].message
        
        print(f"\n📝 Response:")
        print(msg.content[:500] + "..." if len(msg.content) > 500 else msg.content)
        print(f"\n🔧 Tools called: {tools_called if tools_called else 'None'}")
        
        return {"response": msg.content, "tools_called": tools_called}
    
    except Exception as e:
        print(f"\n⚠️ API Error: {e}")
        print("   Falling back to demo mode...")
        return run_demo_agent(level, query)

## Part 3: Test Same Query with Different Agent Levels

In [ ]:
# Test query: Customer reports potential fraud
query = """I think there's a fraudulent charge on my account. 
I see a $299.99 charge from AMZN MKTP US that I didn't make. 
My account ID is ACC-12345. Can you help me?"""

# Run with each agent level
results = {}
for level in [1, 2, 3]:
    results[level] = run_agent(level, query)

In [ ]:
# Compare results
print("\n" + "="*60)
print("COMPARISON: Same Query, Different Agent Levels")
print("="*60)

for level, result in results.items():
    agent = create_agent(level)
    print(f"\nLevel {level} ({agent['name']}):")
    print(f"   Tools called: {result['tools_called']}")
    
    if "initiate_dispute" in result['tools_called']:
        action = "✅ Initiated dispute automatically"
    elif "create_support_ticket" in result['tools_called']:
        action = "📝 Created support ticket for human review"
    elif result['tools_called']:
        action = "👀 Looked up information only"
    else:
        action = "💬 Provided guidance only"
    
    print(f"   Action: {action}")

## Key Insight: Agent Autonomy Levels

| Level | Name | Capabilities | Risk Level |
|-------|------|--------------|------------|
| 1 | Informer | Read-only queries | Low |
| 2 | Advisor | Read + Create tickets | Medium |
| 3 | Operator | Read + Write + Financial actions | High |

**Banking Best Practice:** Match agent autonomy to task risk. High-value transactions should require human approval.

## 🎯 Exercise: Create a Level 4 Agent

Create an "Executor" agent that can:
- Do everything Level 3 can do
- Also process refunds up to $100 automatically
- But requires confirmation for amounts over $100

In [ ]:
# TODO: Add a new tool for processing refunds
refund_tool = {
    "type": "function",
    "function": {
        "name": "process_refund",
        "description": "Process a refund to customer account (max $100 auto-approval)",
        "parameters": {
            "type": "object",
            "properties": {
                "account_id": {"type": "string", "description": "Customer account ID"},
                "amount": {"type": "number", "description": "Refund amount"},
                "reason": {"type": "string", "description": "Reason for refund"}
            },
            "required": ["account_id", "amount", "reason"]
        }
    }
}

# Level 4 tools = Level 3 + refund capability
tools_executor = tools_financial + [refund_tool]

print(f"✅ Level 4 tools defined: {len(tools_executor)} tools")
print("   Includes: get_account_balance, get_transaction_history,")
print("             create_support_ticket, initiate_dispute, process_refund")

---
## ✅ Lab 2 Complete!

**Key Takeaways:**
- Agent autonomy should match the risk level of the task
- Use tool permissions to enforce boundaries
- Same query can have very different outcomes based on agent level
- Always have human escalation paths for high-risk actions

**Next:** Open `03_planning_patterns.ipynb`